<a href="https://colab.research.google.com/github/abubakarkhanlakhwera/Langchain/blob/main/vector_db/vector_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --quiet langchain langchain_community langchain_qdrant qdrant_client langchain_huggingface pypdf openai langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.0 MB/s eta 0:00:00


In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_core.documents import Document
import openai
import os

In [8]:
loader  = PyPDFLoader("/content/WISEWORK AI AGENT PROJECT OVERVIEW.pdf")
documents = loader.load()

In [10]:
len(documents)

4

In [39]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
pages = text_splitter.split_documents(documents)

In [40]:
len(pages)

139

In [41]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [50]:
from datetime import datetime

In [51]:
now = datetime.now()

# Extract only the date
current_date = now.date()

In [63]:
doc_list=[]
for page in pages:
  pg_split= text_splitter.split_text(page.page_content)
  for pg_sub_split in pg_split:
    metadata = {'source':'AI policy','page_no':page.metadata['page']+1,'abubakar':'lakhwera','date':datetime.now().isoformat()}
    doc_string = Document(page_content=pg_sub_split,metadata=metadata)
    doc_list.append(doc_string)

In [64]:
len(doc_list)

139

In [65]:
doc_list[10]

Document(metadata={'source': 'AI policy', 'page_no': 1, 'abubakar': 'lakhwera', 'date': '2025-02-28T15:16:38.665025'}, page_content='to the client explaining Wisework’s services.')

# **qdrant**

In [66]:
qdrant_url = "https://bd4e1587-ec02-453f-b686-77a9b8be1b0c.europe-west3-0.qcp.cloud.qdrant.io:6333"
qdrant_api_key = "_ggh9v-Exxw40Sw0iDpi65Yc3UQ8JeL9fVPznP5NyqPR0-SPU5e _- g"
collection_name = 'AI_policy_new'
client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)


In [ ]:
qdrant = QdrantVectorStore.from_documents(
    doc_list,
    embed_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name=collection_name,
)

# **chroma**

In [68]:
!pip install --quiet langchain_chroma chromadb

In [69]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(doc_list,embed_model , persist_directory="./chroma_db")


In [70]:
vectorstore.persist()

<ipython-input-70-ccfd33473261>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [71]:
query = "Tell me about AI"
results = vectorstore.similarity_search(query, k=2)

In [73]:
for doc in results:
  print(f'Matching Document:{doc}')

Matching Document:page_content='AI.' metadata={'abubakar': 'lakhwera', 'date': '2025-02-28T15:16:38.665702', 'page_no': 1, 'source': 'AI policy'}
Matching Document:page_content='the AI shares:' metadata={'abubakar': 'lakhwera', 'date': '2025-02-28T15:16:38.667401', 'page_no': 2, 'source': 'AI policy'}


# **pine cone**

In [75]:
!pip install --quiet langchain_pinecone pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
from langchain_pinecone import PineconeVectorStore as lang_pinecone
import os
os.environ['PINE_API_KEY ']=PINE_API_KEY

In [ ]:
vector= lang_pinecone.from_documents(doc_list,embed_model,index_name='ai_policy')

In [ ]:
query = 'What is ai policy for student?'

In [ ]:
pinecone_results